In [1]:
#This code is to implement RNN (Recurrent Neural Networks for sentiment analysis)
from keras.layers.recurrent import LSTM
from keras.layers import GRU

from keras.layers.core import Dense, Dropout, SpatialDropout1D, Activation
from keras.layers.convolutional import Conv1D
from keras.layers.embeddings import Embedding
from keras.layers.pooling import GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import collections
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import gensim, logging
from keras.optimizers import SGD
from keras.preprocessing import sequence

np.random.seed(42)

/Users/taiwoalabi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
INPUT_FILE = "/Users/taiwoalabi/Downloads/winemag-data-130k-v2.csv"

In [3]:
Marie= pd.read_csv(INPUT_FILE)

In [4]:
#Marie["Feature1"] = []
for ii in range(len(Marie)):
    
    Marie.loc[ii,"Feature1"] = str(Marie.loc[ii,"description"]) + "," + str(Marie.loc[ii,"country"]) + "," + str(Marie.loc[ii,"province"]) \
    + str(Marie.loc[ii,"variety"]) + "," + str(Marie.loc[ii,"winery"])

In [5]:
VOCAB_SIZE = 5000
EMBED_SIZE = 500
NUM_FILTERS = 1024
NUM_WORDS = 70
BATCH_SIZE = 100
NUM_EPOCHS = 10

In [6]:
counter = collections.Counter()
maxlen = 0
for iii in range(len(Marie)):
    sentence = Marie['Feature1'][iii]
    words = [x.lower() for x in nltk.word_tokenize(sentence)]
    
    if len(words) > maxlen:
        maxlen = len(words)
    for word in words:
        counter[word] += 1

In [7]:
word2index = collections.defaultdict(int)
for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
    word2index[word[0]] = wid + 1
vocab_size = len(word2index) + 1
index2word = {v:k for k, v in word2index.items()}

In [8]:
for ii in range(len(Marie)):
    Marie.loc[ii,"Points1"] = int(Marie.loc[ii,"points"])/100

In [9]:
xs, ys = [], []

for iii in range(len(Marie)):
    sentence = Marie['Feature1'][iii]
    label = float(Marie['Points1'][iii])
    words = [x.lower() for x in nltk.word_tokenize(sentence)]
    ys.append((label))
    wids = [word2index[word] for word in words]
    xs.append(wids)

X = pad_sequences(xs, maxlen=maxlen)




In [10]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, ys, test_size=0.3, random_state=50)

In [11]:
#Bidirectional LSTM improve accuracy
from keras.layers.wrappers import Bidirectional
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.constraints import maxnorm



HIDDEN_LAYER_SIZE = 500




model = Sequential()
model.add(Embedding(vocab_size, EMBED_SIZE,input_length=maxlen))
model.add(SpatialDropout1D(0.3))
model.add(GRU(HIDDEN_LAYER_SIZE, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(HIDDEN_LAYER_SIZE, return_sequences=False))
#model.add(TimeDistributed(Dense(1)))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(1))


print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 162, 500)          2500500   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 162, 500)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 162, 500)          1501500   
_________________________________________________________________
dropout_1 (Dropout)          (None, 162, 500)          0         
_________________________________________________________________
gru_2 (GRU)                  (None, 500)               1501500   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               256512    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
__________

In [12]:
from keras import optimizers
NUM_EPOCHS = 10
lr = 0.001
decay = lr/NUM_EPOCHS
sgd = optimizers.SGD(lr= lr, decay=decay, momentum = 0.9, nesterov = False)

In [ ]:
import keras.backend as K
def RawDiff(y_true, y_pred):
    return ((y_pred)- (y_true))

In [ ]:
model.compile(loss="mean_squared_error", optimizer=sgd, metrics=["mae", "acc", RawDiff])
history = model.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, validation_data=(Xtest, Ytest))

scores = model.evaluate(Xtest, Ytest, verbose=0)
print("Accuracy: %.2f%%" % (scores[2]*100))

Train on 90979 samples, validate on 38992 samples
Epoch 1/10
90979/90979 [==============================] - 6168s 68ms/step - loss: 0.0196 - mean_absolute_error: 0.0819 - acc: 1.5388e-04 - RawDiff: -0.0272 - val_loss: 9.6924e-04 - val_mean_absolute_error: 0.0251 - val_acc: 1.0259e-04 - val_RawDiff: -0.0044
Epoch 2/10
90979/90979 [==============================] - 6173s 68ms/step - loss: 0.0032 - mean_absolute_error: 0.0452 - acc: 1.6487e-04 - RawDiff: -0.0050 - val_loss: 9.5288e-04 - val_mean_absolute_error: 0.0249 - val_acc: 1.0259e-04 - val_RawDiff: -0.0050
Epoch 3/10
90979/90979 [==============================] - 6116s 67ms/step - loss: 0.0023 - mean_absolute_error: 0.0379 - acc: 1.6487e-04 - RawDiff: -0.0032 - val_loss: 9.5326e-04 - val_mean_absolute_error: 0.0249 - val_acc: 1.0259e-04 - val_RawDiff: -0.0057
Epoch 4/10
90979/90979 [==============================] - 6485s 71ms/step - loss: 0.0018 - mean_absolute_error: 0.0342 - acc: 1.6487e-04 - RawDiff: -0.0023 - val_loss: 9.4413e-

In [ ]:
plt.subplot(212)
plt.title("Loss")
plt.plot(history.history["loss"], color="g", label="Train")
plt.plot(history.history["val_loss"], color="b", label="Validation")
plt.legend(loc="best")


plt.subplot(211)
plt.title("Mean_Absolute_Error")
plt.plot(history.history["mean_absolute_error"], color="g", label="Train")
plt.plot(history.history["val_mean_absolute_error"], color="b", label="Validation")
plt.legend(loc="best")

#plt.tight_layout()
plt.show()